In [3]:
# 23:02
# 23:18

import numpy as np
import random
import math


In [12]:
def genDat(n,l):
    rotL = lambda x: x[1:] + [x[0]]
    rand = lambda y: map(lambda z:random.choice([0,1]), [0]*y)
    
    x = map(rand, [l]*n)
    y = map(rotL, x)
    
    x = map(np.array, x)
    y = map(np.array, y)
    
    return zip(x,y)


def logsigmoid(z):
    return 1.0/(1 + math.exp(-z))

def dlogsigmoid(z):
    f = logsigmoid
    return f(z) * (1 - f(z))

class NN:
    def __init__(_,ll,lr=0.05):
        _.ll = ll
        _.lr = lr
        _.fA  = np.vectorize(logsigmoid)
        _.dfA = np.vectorize(dlogsigmoid)
        
        _.w , _.b = _.genWB()
        
    def genWB(_):
        w = [np.random.randn(nin,nout) for (nin, nout) in zip(_.ll[:-1], _.ll[1:])]
        b = [np.random.randn(nout)     for nout        in _.ll[1:]]
        
        return w, b
    
    def ff(_,x):
        v  = [None]*len(_.ll)
        vf = [None]*len(_.ll)
        
        v[0] = vf[0] = x
        
        for i in range(len(_.w)):
            v[i+1]  = np.dot(np.transpose(_.w[i]), vf[i]) + _.b[i]
            vf[i+1] = _.fA(v[i+1])
            
        return v,vf
    
    def fb(_,v,vf, y):
        g  = [None] * len(_.w)
        gw = [None] * len(_.w)
        
        g[-1] = (vf[-1] - y) * _.dfA(v[-1])
        
        for i in range(len(_.w)-2, -1, -1):
            g[i] = np.multiply(np.dot(_.w[i+1], g[i+1]), _.dfA(v[i+1]))
            
        for i in range(len(_.w)-1, -1, -1):
            gw[i] = np.transpose(np.outer(g[i], vf[i]))
            
        return gw, g
    
    def trainStep(_,x,y):
        v,vf  = _.ff(x)
        gw,gb = _.fb(v,vf,y)
        
        _.w = _.w - np.multiply(gw,_.lr)
        _.b = _.b - np.multiply(gb,_.lr)
        
        v2,vf2  = _.ff(x)
        
        [e1,e2] = [_.err(vf[-1], y), _.err(vf2[-1], y)]
        
        return [e1, e2]
    
    def err(_, out, y):
        return math.sqrt(sum((out - y)**2))
    
    def train(_, data):
        return [_.trainStep(x,y) for (x,y) in data]
    
    def test(_, data):
        for (x,y) in data:
            v,vf = _.ff(x)
            e    = _.err(vf[-1], y)
            print "x = {}, y = {}, out = {}, e = {}".format(x,y,vf[-1],e)
            

layers = [[3,3,3], [3,5,3], [3,4,5,3]]
dat = genDat(100000,3)

for ll in layers:
    print "ll = {}".format(ll)
    nn = NN(ll)
    
    nn.test(dat[:5])
    print "TRAIN"
    nn.train(dat)
    nn.test(dat[:10])
    
        
            
        

    

ll = [3, 3, 3]
x = [0 1 0], y = [1 0 0], out = [ 0.58145646  0.62112326  0.28205898], e = 0.800331226369
x = [1 1 1], y = [1 1 1], out = [ 0.61248036  0.60004958  0.26493632], e = 0.92219869441
x = [1 1 1], y = [1 1 1], out = [ 0.61248036  0.60004958  0.26493632], e = 0.92219869441
x = [0 0 1], y = [0 1 0], out = [ 0.56196336  0.46372457  0.35832419], e = 0.855447479448
x = [0 1 0], y = [1 0 0], out = [ 0.58145646  0.62112326  0.28205898], e = 0.800331226369
TRAIN
x = [0 1 0], y = [1 0 0], out = [ 0.96101787  0.02987141  0.03866433], e = 0.0625047058077
x = [1 1 1], y = [1 1 1], out = [ 0.98384909  0.97760642  0.97561998], e = 0.0368335365418
x = [1 1 1], y = [1 1 1], out = [ 0.98384909  0.97760642  0.97561998], e = 0.0368335365418
x = [0 0 1], y = [0 1 0], out = [ 0.02287541  0.97898032  0.01875475], e = 0.0362884572135
x = [0 1 0], y = [1 0 0], out = [ 0.96101787  0.02987141  0.03866433], e = 0.0625047058077
x = [1 0 0], y = [0 0 1], out = [ 0.02004442  0.02914535  0.9832431 ], e = 0

KeyboardInterrupt: 